# Lab-1.7: SFT (Supervised Fine-Tuning) 基線模型訓練

**實驗目標**: 訓練監督式微調基線模型，作為 DPO 的參考模型

在進行 DPO 對齊之前，我們需要先訓練一個 SFT 基線模型。這個基線模型將：
1. **作為 DPO 的起始點**: DPO 需要在 SFT 模型基礎上進行偏好優化
2. **提供性能對比基準**: 評估 DPO 對齊效果
3. **確保基本對話能力**: 在偏好優化前具備基本的指令遵循能力

## SFT vs DPO

- **SFT**: 使用監督學習從指令-回應對中學習
- **DPO**: 使用偏好對比學習來優化模型行為
- **流程**: 預訓練模型 → SFT → DPO → 部署

---

## 步驟 1: 環境載入與配置

重新載入之前準備的環境和數據，並配置 PEFT 進行高效微調。

In [ ]:
import torch
import os
import sys
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, 
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_from_disk, Dataset
import numpy as np
from pathlib import Path

# 設置隨機種子
torch.manual_seed(42)
np.random.seed(42)

print('🚀 開始 SFT 基線模型訓練')
print(f'GPU 可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'當前 GPU: {torch.cuda.get_device_name()}')
    print(f'GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## 步驟 2: 載入模型與 Tokenizer

使用與之前相同的模型配置，並準備 PEFT 配置進行高效微調。

In [ ]:
# 模型配置 (與第一個 notebook 保持一致)
MODEL_NAME = 'microsoft/DialoGPT-medium'  # 或 'gpt2'

print(f'📦 載入模型: {MODEL_NAME}')

# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# 載入模型
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map='auto' if torch.cuda.is_available() else 'cpu'
)

print(f'✅ 模型載入成功')
print(f'模型參數量: {model.num_parameters():,}')
print(f'詞彙大小: {len(tokenizer)}')

## 步驟 3: PEFT 配置

使用 LoRA 進行參數高效微調，減少訓練時間和記憶體需求。

In [ ]:
# 配置 LoRA
lora_config = LoraConfig(
    r=16,                        # rank
    lora_alpha=32,              # scaling parameter
    target_modules=['c_attn', 'c_proj', 'c_fc'],  # 針對 GPT-2 架構
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

# 準備模型進行 PEFT
model = get_peft_model(model, lora_config)

# 顯示可訓練參數
model.print_trainable_parameters()

print('✅ PEFT 配置完成')

## 步驟 4: 準備 SFT 訓練數據

從之前的偏好數據中提取指令-回應對，用於監督式微調。

In [ ]:
# 載入之前處理的數據
try:
    dpo_dataset = load_from_disk('./dpo_data')
    print(f'✅ 載入 DPO 數據集，樣本數: {len(dpo_dataset)}')
except:
    print('⚠️  無法載入已保存的數據，使用模擬數據')
    # 創建模擬訓練數據
    mock_data = [
        {
            'prompt': '請解釋什麼是機器學習?',
            'chosen': '機器學習是人工智能的一個分支，它讓計算機能夠從數據中學習並做出決策，而無需明確編程。通過算法分析大量數據，系統可以識別模式並提高性能。',
            'rejected': '機器學習就是讓機器變聰明。'
        },
        {
            'prompt': '如何學習程式設計?',
            'chosen': '學習程式設計建議從基礎語法開始，選擇一門適合的語言如Python，多做練習項目，參與開源專案，並持續學習新技術。實作是最重要的學習方式。',
            'rejected': '學程式就是寫code。'
        }
    ]
    dpo_dataset = Dataset.from_list(mock_data)

print(f'數據集大小: {len(dpo_dataset)}')
print(f'數據欄位: {list(dpo_dataset.features.keys())}')

In [ ]:
def create_sft_dataset(dataset, tokenizer, max_length=512):
    """將偏好數據轉換為 SFT 訓練格式"""
    sft_data = []
    
    for sample in dataset:
        # 使用 chosen 回應作為訓練目標
        conversation = f"Human: {sample['prompt']}\n\nAssistant: {sample['chosen']}"
        
        # Tokenization
        tokens = tokenizer(
            conversation,
            max_length=max_length,
            truncation=True,
            padding=False,
            return_tensors=None
        )
        
        sft_data.append({
            'input_ids': tokens['input_ids'],
            'attention_mask': tokens['attention_mask'],
            'labels': tokens['input_ids'].copy()  # 對於語言模型，labels 等於 input_ids
        })
    
    return Dataset.from_list(sft_data)

# 創建 SFT 訓練數據集
print('🔄 轉換數據為 SFT 格式...')
sft_dataset = create_sft_dataset(dpo_dataset, tokenizer)

print(f'✅ SFT 數據集準備完成，樣本數: {len(sft_dataset)}')

# 顯示示例
sample = sft_dataset[0]
decoded_text = tokenizer.decode(sample['input_ids'])
print(f'\n📝 訓練樣本示例:')
print(decoded_text[:200] + '...' if len(decoded_text) > 200 else decoded_text)

## 步驟 5: 訓練配置

設置訓練參數，使用適合單 GPU 環境的配置。

In [ ]:
# 訓練配置
output_dir = './sft_model_output'
os.makedirs(output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    
    # 基本設置
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    
    # 優化設置
    warmup_steps=100,
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    
    # 記憶體優化
    fp16=True if torch.cuda.is_available() else False,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    
    # 其他設置
    remove_unused_columns=False,
    report_to=None,  # 不使用 wandb
    seed=42
)

print('⚙️  訓練配置:')
print(f'輸出目錄: {output_dir}')
print(f'訓練輪數: {training_args.num_train_epochs}')
print(f'批次大小: {training_args.per_device_train_batch_size}')
print(f'學習率: {training_args.learning_rate}')
print(f'使用 FP16: {training_args.fp16}')

## 步驟 6: 開始 SFT 訓練

使用 Hugging Face Trainer 進行監督式微調。

In [ ]:
# 數據整理器
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # 不使用 masked language modeling
    pad_to_multiple_of=8
)

# 創建 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sft_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print('🚀 開始 SFT 訓練...')

# 訓練前的記憶體狀態
if torch.cuda.is_available():
    print(f'訓練前 GPU 記憶體: {torch.cuda.memory_allocated() / 1e9:.2f} GB')

# 開始訓練
try:
    trainer.train()
    print('✅ SFT 訓練完成！')
    
    # 訓練後的記憶體狀態
    if torch.cuda.is_available():
        print(f'訓練後 GPU 記憶體: {torch.cuda.memory_allocated() / 1e9:.2f} GB')
        
except Exception as e:
    print(f'❌ 訓練過程中出錯: {e}')
    print('嘗試減少批次大小或序列長度')

## 步驟 7: 保存模型與評估

保存訓練好的 SFT 模型，並進行簡單的生成測試。

In [ ]:
# 保存模型
print('💾 保存 SFT 模型...')

# 保存 PEFT 模型
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f'✅ 模型已保存至: {output_dir}')

# 列出保存的檔案
saved_files = list(Path(output_dir).glob('*'))
print(f'保存的檔案: {[f.name for f in saved_files]}')

In [ ]:
# 測試生成
def test_generation(model, tokenizer, prompt, max_length=100):
    """測試模型生成能力"""
    model.eval()
    
    # 準備輸入
    inputs = tokenizer(prompt, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # 生成回應
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 解碼輸出
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated_text[len(prompt):].strip()
    
    return response

# 測試生成
print('🧪 測試 SFT 模型生成...')

test_prompts = [
    "Human: 什麼是深度學習?\n\nAssistant:",
    "Human: 如何提高程式設計技能?\n\nAssistant:"
]

for i, prompt in enumerate(test_prompts):
    print(f'\n📝 測試 {i+1}:')
    print(f'Prompt: {prompt.split("Assistant:")[0]}Assistant:')
    
    try:
        response = test_generation(model, tokenizer, prompt)
        print(f'Generated: {response}')
    except Exception as e:
        print(f'生成失敗: {e}')
    
    print('-' * 60)

## 步驟 8: 訓練總結

總結 SFT 基線模型的訓練結果。

In [ ]:
# 訓練總結
print('=== SFT 基線模型訓練總結 ===')
print(f'✅ 基礎模型: {MODEL_NAME}')
print(f'✅ PEFT 方法: LoRA (r={lora_config.r}, alpha={lora_config.lora_alpha})')
print(f'✅ 訓練數據: {len(sft_dataset)} 個樣本')
print(f'✅ 訓練輪數: {training_args.num_train_epochs}')
print(f'✅ 模型保存位置: {output_dir}')

# 顯示訓練參數統計
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f'✅ 可訓練參數: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)')

print('\n🎯 SFT 基線模型已準備完成！')
print('下一步: 執行 03-DPO_Training.ipynb 進行偏好優化')

# 清理 GPU 記憶體
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f'GPU 記憶體已清理，當前使用: {torch.cuda.memory_allocated() / 1e9:.2f} GB')